In [3]:
import numpy as np
import pandas as pd
from keras.utils import image_dataset_from_directory

In [4]:
df="/kaggle/input/landscape-image-colorization/landscape Images"

In [7]:
train_data=image_dataset_from_directory(
df,
image_size=(256,256),
batch_size=32,
shuffle=True,
seed=123,
label_mode=None
)

Found 14258 files.


# # GAN

In [18]:
from keras.layers import Dense,Conv2D,MaxPooling2D,Conv2DTranspose,BatchNormalization,Dropout,Input,Reshape,ReLU,Flatten
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [35]:
early_stopping=EarlyStopping(
monitor="val_loss",
patience="val_loss",
verbose=1)

In [19]:
def build_gen():
    inputs=Input(shape=(100,))
    C=Dense(16*16*256)(inputs)
    C=Reshape((16,16,256))(C)
    C=Conv2DTranspose(256,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=Conv2DTranspose(128,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)

    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=Conv2DTranspose(64,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)                                                                                                                    
    C=Conv2DTranspose(32,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
   
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)                       
    outputs=Conv2D(3,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01),activation="tanh")(C)
    model=Model(inputs,outputs)
    return model
gen=build_gen()
gen.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 65536)          │     6,619,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_5              │ (None, 32, 32, 256)    │       590,080 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_12 (ReLU)                 │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_6              │ (None, 64, 64, 128)    │       295,040 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_13 (ReLU)                 │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_7              │ (None, 128, 128, 64)   │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_14 (ReLU)                 │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_8              │ (None, 256, 256, 32)   │        18,464 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_15 (ReLU)                 │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 256, 256, 3)    │           867 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,599,299 (28.99 MB)

 Trainable params: 7,598,339 (28.99 MB)

 Non-trainable params: 960 (3.75 KB)

In [20]:
def build_dis():
    inputs=Input(shape=(256,256,3))
    C=Conv2D(32,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
    C=Conv2D(128,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)

    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)
    C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)

    C=Conv2D(64,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)      
    C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)

    C=Conv2D(32,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
    C=ReLU()(C)
   
    C=Dropout(0.5)(C)
    C=BatchNormalization()(C)       
    C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
    C=Flatten()(C)
    D=Dense(32,activation="relu")(C)
    outputs=Dense(1,activation="sigmoid")(D)
    model=Model(inputs,outputs)
    return model
dis=build_dis()
dis.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_16 (ReLU)                 │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 128, 128, 128)  │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_17 (ReLU)                 │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 128, 128, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 64, 64, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_18 (ReLU)                 │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 32, 32, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_19 (ReLU)                 │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │       262,17

 Total params: 393,377 (1.50 MB)

 Trainable params: 392,865 (1.50 MB)

 Non-trainable params: 512 (2.00 KB)

In [24]:
def build_gan(gen,dis):
    dis.trainable=False
    inputs=Input(shape=(100,))
    fake_img=gen(inputs)
    outputs=dis(fake_img)
    model=Model(inputs,outputs)
    return model
gan=build_gan(gen,dis)
gan.summary()


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_5 (Functional)       │ (None, 256, 256, 3)    │     7,599,299 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_7 (Functional)       │ (None, 1)              │       393,377 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,992,676 (30.49 MB)

 Trainable params: 7,598,339 (28.99 MB)

 Non-trainable params: 394,337 (1.50 MB)

In [42]:
def train_gan(gan,gen,dis,epochs,batch_size,latent_dim,dataset):
    dis.trainable=True
    dis.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])
    gan.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
    for epoch in range(epochs):
        for real_img in dataset:
            real_label=np.ones((batch_size,1))
            fake_label=np.zeros((batch_size,1))
            noise=np.random.normal(0,1,(batch_size,100))
            fake_img=gen.predict(noise)
            dis.trainable=True
            dis.train_on_batch(real_img,real_label)
            dis.train_on_batch(fake_img,fake_label)
            dis.trainable=False
            g_loss=gan.train_on_batch(noise,real_label)
            return gan
train_gan=train_gan(gan,gen,dis,2,32,100,train_data)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [46]:
noise=np.random.normal(0,1,(32,100))
train_gan.predict(noise)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


array([[0.43020973],
       [0.42965147],
       [0.4295345 ],
       [0.4301134 ],
       [0.43007028],
       [0.42981568],
       [0.43002084],
       [0.42993078],
       [0.4298835 ],
       [0.4302921 ],
       [0.4299682 ],
       [0.43018982],
       [0.42975605],
       [0.4298119 ],
       [0.43014833],
       [0.42972046],
       [0.43024832],
       [0.42980406],
       [0.43030906],
       [0.4296956 ],
       [0.43025824],
       [0.43024424],
       [0.4300135 ],
       [0.42984363],
       [0.4299946 ],
       [0.4303547 ],
       [0.4301964 ],
       [0.43019068],
       [0.4295824 ],
       [0.42985794],
       [0.429921  ],
       [0.42960167]], dtype=float32)

# # AutoEncoder

In [34]:
inputs=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
C=ReLU()(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)
C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
C=Conv2D(64,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=ReLU()(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)
C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
C=Conv2D(128,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=ReLU()(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)      
C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)
C=Conv2D(256,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=ReLU()(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)       
C=MaxPooling2D(pool_size=(2,2),strides=(2,2))(C)

C=Conv2D(556,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)


C=Conv2DTranspose(256,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=ReLU()(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)
C=Conv2DTranspose(128,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=ReLU()(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)
C=Conv2DTranspose(64,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=ReLU()(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C) 
C=Conv2DTranspose(32,(3,3),strides=(2,2),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(C)
C=ReLU()(C)
C=Dropout(0.5)(C)
C=BatchNormalization()(C)                       
outputs=Conv2D(3,(3,3),padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01),activation="tanh")(C)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["accuracy"])
model.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_28 (ReLU)                 │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_29 (ReLU)                 │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_30 (ReLU)                 │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_31 (ReLU)                 │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_31          │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 16, 16, 556)    │     1,281,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_13             │ (None, 32, 32, 256)    │     1,281,28

 Total params: 3,343,279 (12.75 MB)

 Trainable params: 3,341,359 (12.75 MB)

 Non-trainable params: 1,920 (7.50 KB)

In [50]:
for i in train_data:
    model.fit(i,i,epochs=3,batch_size=32)
    break

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step - accuracy: 0.3365 - loss: 825.7640
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.3373 - loss: 815.4295
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - accuracy: 0.3365 - loss: 804.8002


In [55]:
for i in train_data.take(1):
    prediction=model.predict(i)
    reconstruction_error=np.mean(np.square(i-prediction))
    threshold=np.mean(reconstruction_error)+2*np.std(reconstruction_error)
    label=np.where(reconstruction_error>threshold,0,1)
    accuracy=np.mean(label==0)*100
    print(accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
0.0
